<a href="https://colab.research.google.com/github/dnhshl/cc-ai/blob/main/Kryptographie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kryptographie

Beispiele u.a. in Anlehnung an [Practical Cryptography for Developers](https://cryptobook.nakov.com/)

## AES Encrypt / Decrypt

In [ ]:
!pip install pycryptodome

In [ ]:
from Crypto.Cipher import AES
import binascii, os

In [ ]:
def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(encryptedMsg, secretKey):
    (ciphertext, nonce, authTag) = encryptedMsg
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

In [ ]:
secretKey = os.urandom(32)  # 256-bit random encryption key
print("Encryption key:", binascii.hexlify(secretKey))

In [ ]:
msg = b'Hello World, how are you today?'

In [ ]:
encryptedMsg = encrypt_AES_GCM(msg, secretKey)
print("encryptedMsg", {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'aesIV': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2])
})

In [ ]:
decryptedMsg = decrypt_AES_GCM(encryptedMsg, secretKey)
print("decryptedMsg", decryptedMsg)

## RSA

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

### RSA Keys

In [ ]:
keyPair = RSA.generate(3072)

pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n)}, e={hex(pubKey.e)})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))


privateKey = keyPair
print(f"Private key: (n={hex(pubKey.n)}, d={hex(keyPair.d)})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))

### RSA Encrypt / Decrypt

In [ ]:
msg = b'A message for encryption'
encryptor = PKCS1_OAEP.new(pubKey)
encrypted = encryptor.encrypt(msg)
print("Encrypted:", binascii.hexlify(encrypted))

In [ ]:
decryptor = PKCS1_OAEP.new(privateKey)
decrypted = decryptor.decrypt(encrypted)
print('Decrypted:', decrypted)

## Hashing

In [ ]:
import hashlib, binascii
from Crypto.Hash import SHA256
from Crypto.Hash import SHA3_256

In [ ]:
text = 'Überweise 100 Euro'
msg = text.encode("utf8")

md5hash = hashlib.md5(msg).digest()
print("MD5:   ", binascii.hexlify(md5hash))

sha256hash = SHA256.new(msg).digest()
print("SHA-256:   ", binascii.hexlify(sha256hash))

sha3_256 = SHA3_256.new(msg).digest()
print("SHA3-256:  ", binascii.hexlify(sha3_256))



In [ ]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

md5hash = hashlib.md5(msg).digest()
print("MD5:   ", binascii.hexlify(md5hash))

sha256hash = SHA256.new(msg).digest()
print("SHA-256:   ", binascii.hexlify(sha256hash))

sha3_256 = SHA3_256.new(msg).digest()
print("SHA3-256:  ", binascii.hexlify(sha3_256))


## MAC

In [ ]:
import hashlib, hmac, binascii

In [ ]:
def hmac_sha256(key, msg):
  return hmac.new(key, msg, hashlib.sha256).digest()

key = b"12345"


In [ ]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

print(binascii.hexlify(hmac_sha256(key, msg)))

## Signatur


In [ ]:
from Crypto.Signature import pkcs1_15


### Alice

In [ ]:
text = 'Überweise 1000 Euro'
msg = text.encode("utf8")

# Hash
hash_obj = SHA3_256.new(msg)
print("SHA-256:   ", binascii.hexlify(hash_obj.digest()))

# Encrypt Hash with private key
signature = pkcs1_15.new(privateKey).sign(hash_obj)
print("Signature", binascii.hexlify(signature))


### Bob

In [ ]:
msg = msg

# Hash
hash_obj = SHA3_256.new(msg)

# Verify the signature with the public key
try:
    pkcs1_15.new(pubKey).verify(hash_obj, signature)
    print("The signature is valid.")
except (ValueError, TypeError):
    print("The signature is not valid.")

## Verschlüsseln und Signieren

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import binascii

In [ ]:

# Alice generates her RSA key pair
alice_key = RSA.generate(2048)
alice_private_key = alice_key
alice_public_key = alice_key.publickey()

# Bob generates his RSA key pair
bob_key = RSA.generate(2048)
bob_private_key = bob_key
bob_public_key = bob_key.publickey()

### Alice

In [ ]:

# Message to be signed and encrypted
message = b'This is a secret message'

# Alice signs the message with her private key
hash_obj = SHA256.new(message)
signature = pkcs1_15.new(alice_private_key).sign(hash_obj)

# Alice encrypts the message and the signature with Bob's public key
encryptor = PKCS1_OAEP.new(bob_public_key)
encrypted_message = encryptor.encrypt(message)
print("Encrypted Message:", binascii.hexlify(encrypted_message))


### Bob

In [ ]:
# Bob decrypts the received message and signature with his private key
decryptor = PKCS1_OAEP.new(bob_private_key)
decrypted_message = decryptor.decrypt(encrypted_message)
print('Decrypted Message:', decrypted_message)

# Bob verifies the signature with Alice's public key
hash_obj = SHA256.new(decrypted_message)
try:
    pkcs1_15.new(alice_public_key).verify(hash_obj, signature)
    print("The signature is valid.")
except (ValueError, TypeError):
    print("The signature is not valid.")
